<a href="https://colab.research.google.com/github/abhiiyer/EIP3/blob/master/Session3/AbhishekIyer_CycleGAN_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
########### Cell Output has been cleared else GITHUB becomes tough to read since there are lots of images #################

######## Implementation of Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks. ########

### Github link is https://github.com/eriklindernoren/Keras-GAN

# Step-1: Mount the Google Drive on Colab 

from google.colab import drive
drive.mount('/content/drive')

# Step-2: Important is to downgrade scipy to 1.1.0 (!pip install scipy==1.1.0) and restart run-time

!pip install scipy==1.1.0

# Step-3: Download the dataset 

!git clone https://github.com/eriklindernoren/Keras-GAN.git
%cd /content/Keras-GAN/cyclegan
!ls
!bash download_dataset.sh apple2orange
!cp -r datasets/ /content/

# Step-4: Change the directory to root for COLAB (Else the "images" folder are created in CycleGAN directory mistakenly)
%cd /content


#########################################
# Step-5: COLAB Run-time gets disconnected after around 4.5 hrs (Have done trial-and-error)
### When run-time is disconnected, since we are saving the images temporary workspace, those would also be lost 
### Hence we need to save all the images 

## Code to save SINGLE IMAGE in local 
## files.download("/content/Keras-GAN/cyclegan/images/apple2orange/0_200.png")

## Code to save ENTIRE FOLDER 

#from google.colab import files

#For the folder you have to zip it first and can only download later on
#!zip -r images.zip /content/Keras-GAN/cyclegan/images/apple2orange

#Download files
#files.download('images.zip')
#########################################

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
     |████████████████████████████████| 31.2MB 1.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.3.1
    Uninstalling scipy-1.3.1:
      Successfully uninstalled scipy-1.3.1
Cloning into 'Keras-GAN'...
remote: Enumerating objects: 3469, done.
remote: Total 3469 (delta 0), reused 0 (delta 0), pack-reused 3469
Receiving objects: 100% (3469/3469), 

In [0]:
! pwd  ## Check where the "images" folder will get CREATED.

/content


## CYCLEGAN : 
---

## *CycleGAN for Unpaired Image-to-Image Translation*  (*Implemention of CycleGAN for Apple2Orange Dataset*)


## Image to Image Translation Problems 

Image-to-Image translation is a class of computer vision and graphics problems, where the goal is to learn the mapping between an input domain to an output domain.

Image-to-image translation is a general task where an image from one domain is converted to a corresponding image in another domain.

Traditionally different approaches have been proposed depending on whether aligned image pairs or two sets of (unaligned) examples from both domains are available for training. While paired training samples might be difficult to obtain, the unpaired setup leads to a highly under-constrained problem and inferior results.

<a class="anchor" id="2"></a>
## Solving an Image to Image Translation Problems 

Many vision and graphics problems such as converting grayscale to colour images, satellite images to maps, sketches to photographs, and paintings to photographs can be seen as image-to-image translation tasks.

Traditionally,each problem has been tackled with specific architectures and loss functions that do not generalize well to other tasks. Recently, Generative Adversarial Networks (GANs) have provided an alternative approach, where the loss function is learned from the data instead of defining it manually.Impressive results have been demonstrated in several areas such as image inpainting, style transfer,super-resolution and image colorisation. The Image-to-Image problems are overcomed by th below two approaches.

<a class="anchor" id="2.1"></a>
### Using image-to-image translation using paired training samples

Conditional GAN architectures have been widely applied in image-to-image translation tasks where sufficient amount of paired training examples are available.In particular, proposed a general purpose image to image translator "pix2pix" for various applications such as semantic maps to real image, edges to shoe images, and gray to color images. 


<a class="anchor" id="2.2"></a>
### Using image-to-image translation using unpaired training samples

Three concurrent works "cycleGAN", "DiscoGAN" and "DualGAN" propose architectures consisting of two generator-discriminator pairs. The first pair learns a forward mapping from source to target domain, while the second pair learns the inverse
of it. Both pairs are trained simultaneously using adversarial losses and an additional loss that encourages cascaded mapping to reproduce the original input image (referred as cycle consistency loss).

The basic idea of Image to Image Translation is illustrated in the image below:

![Image to Image Translation](https://mikigom.github.io/img/pixcyccompare.png)

Left: Traditional training data, comprising training examples {𝑥𝑖, 𝑦𝑖}𝑖=1 𝑁, where𝑥𝑖 and 𝑦𝑖are an aligned pair.
CycleGAN addresses the situation where paired data is unavailable, it deals with a source set {𝑥} 𝑖=1 𝑁 (𝑥𝑖 ∈ 𝑋) and a target
set {𝑦} 𝑖=1 𝑁 (𝑦𝑖 ∈ 𝑌)(right), with no alignment of specific 𝑥𝑖 -s with specific 𝑦𝑖 -s.


<a class="anchor" id="3"></a>
## Generative Adversarial Network(GAN) Overview 

Generative adversarial networks (GANs), introduced in 2014 by Goodfellow are an alternative to VAEs for learning latent spaces of images. They enable the generation of fairly realistic synthetic images by forcing the generated images to be statistically
almost indistinguishable from real ones.

An intuitive way to understand GANs is to imagine a forger trying to create a fake Picasso painting. At first, the forger is pretty bad at the task. He mixes some of his fakes with authentic Picassos and shows them all to an art dealer. The art dealer makes an authenticity assessment for each painting and gives the forger feedback about what makes a Picasso look like a Picasso. The forger goes back to his studio to prepare some new fakes. As times goes on, the forger becomes increasingly competent at imitating the style of Picasso, and the art dealer becomes increasingly expert at spotting fakes. In the end, they have on their hands some excellent fake Picassos.


The GAN comprises of two networks:

![gan](https://i2.wp.com/www.kdnuggets.com/wp-content/uploads/generative-adversarial-network.png)

<a class="anchor" id="4"></a>
## Unpaired Image-to-Image Translation using CycleGAN

<a class="anchor" id="4.1"></a>
### CycleGAN Overview

CycleGAN is used to image-to-image translation without input-output pairs.In Image-to-image translation, the network need to learn the mapping between an input image and output image using a training set of aligned image pairs. This is a major problem for many applications as paired training data is not available. 

CycleGANs enable learning a mapping from one domain X to another domain Y without having to find 
perfectly matched training pairs. CycleGAN is an approach for learning to translate an image between domains in the absence of paired examples.

The CycleGAN algorithm learns to automatically “translate” an image between two “domains”, by training on two unordered image collections.

![cycleGAN image](https://junyanz.github.io/CycleGAN/images/teaser.jpg)

(left) Monet painting and landscape photo from Flickr; (center) zebras and horse from ImageNet; (right) summer and winter Yosemite photos from Flickr.

The novelty of CycleGAN is its ability to learn to transform between domains without one-to-one paired training data from source and target domains. Most existing methods rely on paired examples for training. CycleGAN overcomes the need for a corresponding image in the target domain by a two-step transformation of source domain image-first mapping it to target domain and then back to the original image.

CycleGAN employs Cycle Consistency using transitivity as a way to gauge successful translation. A generator network mapping the image to target domain is pitted against an adversarial discriminator network to improve the quality of generated images.

#### Discriminator Network:

Takes as input an image (real or synthetic), and predicts whether the image came from the training set or was created by the generator network. 

Discriminator Network evaluates the authenticity of each data instance. The discriminator decides whether each instance of data it reviews belongs to the actual training dataset or Generated fake data.

Discriminator Network interprets and differentiates whether the input data is Real or Fake.


### Adversarial Networks

We have a generator network and discriminator network playing against each other. The generator tries to produce samples from the desired distribution and the discriminator tries to predict if the sample is from the actual distribution or produced by the generator. The generator and discriminator are trained jointly. The effect this has is that eventually the generator learns to approximate the underlying distribution completely and the discriminator is left guessing randomly.


### Cycle-Consistent

The above adversarial method of training has a problem though. Quoting the authors of the original paper:
Adversarial training can, in theory, learn mappings  G  and  F  that produce outputs identically distributed as target domains  Y  and  X  respectively. However, with large enough capacity, a network can map the same set of input images to any random permutation of images in the target domain, where any of the learned mappings can induce an output distribution that matches the target distribution. Thus, an adversarial loss alone cannot guarantee that the learned function can map an individual input  xi  to a desired output  yi .

To regularize the model, the authors introduce the constraint of cycle-consistency - if we transform from source distribution to target and then back again to source distribution, we should get samples from our source distribution.

<a class="anchor" id="5"></a>
### CycleGAN Network Architecture
  

![Cyclicgan Architecture](https://junyanz.github.io/CycleGAN/images/cyclegan_blogs.jpg)

   
In a paired dataset, every image, say  imgA , is manually mapped to some image, say  imgB , in target domain, such that they share various features. Features that can be used to map an image  (imgA/imgB)  to its correspondingly mapped counterpart  (imgB/imgA) . Basically, pairing is done to make input and output share some common features. This mapping defines meaningful transformation of an image from one damain to another domain. So, when we have paired dataset, generator must take an input, say  inputA , from domain  DA  and map this image to an output image, say  genB , which must be close to its mapped counterpart. But we don't have this luxury in unpaired dataset, there is no pre-defined meaningful transformation that we can learn, so, we will create it. We need to make sure that there is some meaningful relation between input image and generated image. So, authors tried to enforce this by saying that Generator will map input image  (inputA)  from domain  DA  to some image in target domain  DB , but to make sure that there is meaningful relation between these images, they must share some feature, features that can be used to map this output image back to input image, so there must be another generator that must be able to map back this output image back to original input. So, you can see this condition defining a meaningful mapping between  inputA  and  genB .

In a nutshell, the model works by taking an input image from domain  DA  which is fed to our first generator  GeneratorA→B  whose job is to transform a given image from domain  DA  to an image in target domain  DB . This new generated image is then fed to another generator  GeneratorB→A  which converts it back into an image,  CyclicA , from our original domain  DA  (think of autoencoders, except that our latent space is  Dt ). And as we discussed in above paragraph, this output image must be close to original input image to define a meaningful mapping that is absent in unpaired dataset.

As you can see in above figure, two inputs are fed into each discriminator(one is original image corresponding to that domain and other is the generated image via a generator) and the job of discriminator is to distinguish between them, so that discriminator is able to defy the adversary (in this case generator) and reject images generated by it. While the generator would like to make sure that these images get accepted by the discriminator, so it will try to generate images which are very close to original images in Class  DB . (In fact, the generator and discriminator are actually playing a game whose Nash equilibrium is achieved when the generator's distribution becomes same as the desired distribution)

In [0]:
import scipy
from glob import glob
import numpy as np

class DataLoader():
  
    def __init__(self, dataset_name, img_res=(128, 128)):
        #print("Inside the constructor")
        print("dataset_name is ",dataset_name)
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
      
        #print("Inside load_data() method")
        
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        
        #print("dataset_name in load_data() is ",self.dataset_name)
        #print("data_type in load_data() is ",data_type)
        path = glob('/content/datasets/%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        #print("Inside load_batch() method")
        data_type = "train" if not is_testing else "val"
        path_A = glob('/content/datasets/%s/%sA/*' % (self.dataset_name, data_type))
        path_B = glob('/content/datasets/%s/%sB/*' % (self.dataset_name, data_type))

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [0]:
from keras.layers import Layer, InputSpec
from keras import initializers, regularizers, constraints
from keras import backend as K


class InstanceNormalization(Layer):
    """Instance normalization layer.
    Normalize the activations of the previous layer at each step,
    i.e. applies a transformation that maintains the mean activation
    close to 0 and the activation standard deviation close to 1.
    # Arguments
        axis: Integer, the axis that should be normalized
            (typically the features axis).
            For instance, after a `Conv2D` layer with
            `data_format="channels_first"`,
            set `axis=1` in `InstanceNormalization`.
            Setting `axis=None` will normalize all values in each
            instance of the batch.
            Axis 0 is the batch dimension. `axis` cannot be set to 0 to avoid errors.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
            When the next layer is linear (also e.g. `nn.relu`),
            this can be disabled since the scaling
            will be done by the next layer.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a Sequential model.
    # Output shape
        Same shape as input.
    # References
        - [Layer Normalization](https://arxiv.org/abs/1607.06450)
        - [Instance Normalization: The Missing Ingredient for Fast Stylization](
        https://arxiv.org/abs/1607.08022)
    """
    def __init__(self,
                 axis=None,
                 epsilon=1e-3,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):
        ndim = len(input_shape)
        if self.axis == 0:
            raise ValueError('Axis cannot be zero')

        if (self.axis is not None) and (ndim == 2):
            raise ValueError('Cannot specify axis for rank 1 tensor')

        self.input_spec = InputSpec(ndim=ndim)

        if self.axis is None:
            shape = (1,)
        else:
            shape = (input_shape[self.axis],)

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.gamma_constraint)
        else:
            self.gamma = None
        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constraint)
        else:
            self.beta = None
        self.built = True

    def call(self, inputs, training=None):
        input_shape = K.int_shape(inputs)
        reduction_axes = list(range(0, len(input_shape)))

        if self.axis is not None:
            del reduction_axes[self.axis]

        del reduction_axes[0]

        mean = K.mean(inputs, reduction_axes, keepdims=True)
        stddev = K.std(inputs, reduction_axes, keepdims=True) + self.epsilon
        normed = (inputs - mean) / stddev

        broadcast_shape = [1] * len(input_shape)
        if self.axis is not None:
            broadcast_shape[self.axis] = input_shape[self.axis]

        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            normed = normed * broadcast_gamma
        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            normed = normed + broadcast_beta
        return normed

    def get_config(self):
        config = {
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constraint': constraints.serialize(self.beta_constraint),
            'gamma_constraint': constraints.serialize(self.gamma_constraint)
        }
        base_config = super(InstanceNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

Using TensorFlow backend.


In [0]:
from __future__ import print_function, division
import scipy
from google.colab import files

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

In [0]:
 # Input shape
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)

 # Configure data loader
dataset_name = 'apple2orange'
data_loader = DataLoader(dataset_name=dataset_name, img_res=(img_rows, img_cols))  

dataset_name is  apple2orange


In [0]:
 # Calculate output shape of D (PatchGAN)
patch = int(img_rows / 2**4)
disc_patch = (patch, patch, 1)
print(disc_patch)

# Number of filters in the first layer of G and D
gf = 32
df = 64

 # Loss weights
lambda_cycle = 10.0                    # Cycle-consistency loss
lambda_id = 0.1 * lambda_cycle    # Identity loss
optimizer = Adam(0.0002, 0.5)

(8, 8, 1)


In [0]:
def conv2d(layer_input, filters, f_size=4):
#Layers used during downsampling
    d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    d = InstanceNormalization()(d)
    return d

In [0]:
def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
#Layers used during upsampling
    u = UpSampling2D(size=2)(layer_input)
    u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
    if dropout_rate:
        u = Dropout(dropout_rate)(u)
    u = InstanceNormalization()(u)
    u = Concatenate()([u, skip_input])
    return u


### Building Generator

First we will build the a class representing our generators, which are comprised of 3 main components: 

![Generator](https://hardikbansal.github.io/CycleGANBlog/images/Generator.jpg)

1) Encoder

2) Transformer 

3) Decoder.

The Generator model is based on the U-net architecture.


#### Encoder
 
 In the encoding step of the generator, what we're doing is essentially learning the features of an image using a convolutional neural network (CNN). Intuitively, CNNs learn features of an image by examining the spatial relationships between smaller regions by applying kernels (or filters) on the original image. 
 
The input size [256,256,3]is reshaped to [128,128,3] . The first step is extracting the features from an image which is done a convolution network. As input a convolution network takes an image, size of filter window that we move over input image to extract out features and the stride size to decide how much we will move filter window after each step. So the first layer of encoding looks like this:

#### Decoder
Very similar in structure to the encoder, the decoder reverses the effects of the encoder to generate an image in our target domain G(x) from its feature vector. 

In [0]:
def build_generator():
#U-Net Generator      

# Image input
    d0 = Input(shape=img_shape)

# Downsampling
    d1 = conv2d(d0, gf)
    d2 = conv2d(d1, gf*2)
    d3 = conv2d(d2, gf*4)
    d4 = conv2d(d3, gf*8)

# Upsampling
    u1 = deconv2d(d4, d3, gf*4)
    u2 = deconv2d(u1, d2, gf*2)
    u3 = deconv2d(u2, d1, gf)

    u4 = UpSampling2D(size=2)(u3)
    output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

    return Model(d0, output_img)

In [0]:
def d_layer(layer_input, filters, f_size=4, normalization=True):
#Discriminator layer
    d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    if normalization:
        d = InstanceNormalization()(d)
    return d	

<a class="anchor" id="6.2"></a>
### Building Descriminator 

Using the general purpose functions above, we can also define our discriminator network, which is simply a series of CNN layers that ultimately reduces the output to 1 dimensional prediction (real/fake).

![Descriminator](https://image.slidesharecdn.com/kaonetv2-facetranslation-171029102202/95/kaonet-v2-face-translation-using-cyclegan-25-638.jpg?cb=1509275338)

The discriminator is simply a convolution network in our case. First, we will extract the features from the image

In [0]:
def build_discriminator():       

    img = Input(shape=img_shape)

    d1 = d_layer(img, df, normalization=False)
    d2 = d_layer(d1, df*2)
    d3 = d_layer(d2, df*4)
    d4 = d_layer(d3, df*8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)
    return Model(img, validity)	

We now have two main components of the model, namely Generator and Discriminator, and since we want to make this model work in both the direction i.e., from A→B and from B→A , we will have two Generators, namely GeneratorA→B and GeneratorB→A , and two Discriminators, namely DiscriminatorA and DiscriminatorB .


#### Discriminator loss

Discriminator must be trained such that recommendation for images from category A must be as close to 1, and vice versa for discriminator B. So Discriminator A would like to minimize  (DiscriminatorA(a)−1)2  and same goes for B as well. This can be implemented as:

In [0]:
# Build and compile the discriminators
d_A = build_discriminator()
d_B = build_discriminator()
print(d_A.summary())

d_A.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])
d_B.compile(loss='mse',optimizer=optimizer, metrics=['accuracy'])

W0827 13:24:24.401515 140242092234624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 13:24:24.403932 140242092234624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 13:24:24.410012 140242092234624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0827 13:24:24.858004 140242092234624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        3136      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       131200    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
instance_normalization_1 (In (None, 32, 32, 128)       2         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       524544    
__________

In [0]:
# Build the generators
g_AB = build_generator()
g_BA = build_generator()
print(g_AB.summary())

W0827 13:24:25.066577 140242092234624 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 64, 64, 32)   1568        input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_9 (LeakyReLU)       (None, 64, 64, 32)   0           conv2d_11[0][0]                  
__________________________________________________________________________________________________
instance_normalization_7 (Insta (None, 64, 64, 32)   2           leaky_re_lu_9[0][0]              
__________________________________________________________________________________________________
conv2d_12 

In [0]:
# Input images from both domains
img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)

In [0]:
 # Translate images to the other domain
fake_B = g_AB(img_A)
fake_A = g_BA(img_B)

In [0]:
 # Translate images back to original domain
reconstr_A = g_BA(fake_B)
reconstr_B = g_AB(fake_A)

In [0]:
#Identity mapping of images
img_A_id = g_BA(img_A)
img_B_id = g_AB(img_B)

In [0]:
# For the combined model we will only train the generators
d_A.trainable = False
d_B.trainable = False

In [0]:
# Discriminators determines validity of translated images
valid_A = d_A(fake_A)
valid_B = d_B(fake_B)

<a class="anchor" id="7"></a>
### Loss Function
we have two generators and two discriminators. We need to design the loss function in a way which accomplishes our goal. The loss function can be seen having four parts:

1) Discriminator must approve all the original images of the corresponding categories.

2) Discriminator must reject all the images which are generated by corresponding Generators to fool them.

3) Generators must make the discriminators approve all the generated images, so as to fool them.

4) The generated image must retain the property of original image, so if we generate a fake image using a generator say  GeneratorA→B  then we must be able to get back to original image using the another generator  GeneratorB→A  - it must satisfy cyclic-consistency.

In [0]:
# Combined model trains generators to fool discriminators
combined = Model(inputs=[img_A, img_B],outputs=[ valid_A, valid_B,reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])

In [0]:
combined.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
model_4 (Model)                 (None, 128, 128, 3)  1545425     input_6[0][0]                    
                                                                 model_3[1][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
model_3 (M

#### Combined loss
Generator should eventually be able to fool the discriminator about the authencity of it's generated images. This can done if the recommendation by discriminator for the generated images is as close to 1 as possible. So generator would like to minimize  (DiscriminatorB(GeneratorA→B(a))−1)2  So the loss is:


In [0]:
combined.compile(loss=['mse', 'mse','mae', 'mae','mae', 'mae'],
                            loss_weights=[  1, 1,lambda_cycle, lambda_cycle,lambda_id, lambda_id ],
                            optimizer=optimizer)

In [0]:
imageSize = 128
from PIL import Image
from IPython.display import display
def showX(X, rows=1):
    assert X.shape[0]%rows == 0
    int_X = ( (X+1)/2*255).clip(0,255).astype('uint8')    
    int_X = int_X.reshape(-1,imageSize,imageSize, 3)
    int_X = int_X.reshape(rows, -1, imageSize,imageSize,3).swapaxes(1,2).reshape(rows*imageSize,-1, 3)
    display(Image.fromarray(int_X))

In [0]:
def sample_images(epoch, batch_i):
  
    #print("Inside sample_images() function ")
    #print("Epoch is  ",epoch)
    os.makedirs('images/%s' % dataset_name, exist_ok=True)
    r, c = 2, 3

    ## Added 
    data_loader = DataLoader(dataset_name,img_res=(128, 128))
    
    imgs_A = data_loader.load_data(domain="A", batch_size=1, is_testing=True)
    imgs_B = data_loader.load_data(domain="B", batch_size=1, is_testing=True)

    # Translate images to the other domain
    fake_B = g_AB.predict(imgs_A)
    fake_A = g_BA.predict(imgs_B)
   
    # Translate back to original domain
    reconstr_A = g_BA.predict(fake_B)
    reconstr_B = g_AB.predict(fake_A)

    gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    # Show/plot on terminal the results of Both the Networks A->B & B->A
    titles = ['Original', 'Translated', 'Reconstructed']
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i, j].set_title(titles[j])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%s/%d_%d.png" % (dataset_name, epoch, batch_i))
    #files.download("images/%s/%d_%d.png" % (dataset_name, epoch, batch_i))
    plt.close()

In [0]:
def train(epochs, batch_size=1, sample_interval=200):

    print("Train method")
    print("epochs is ",epochs)
    start_time = datetime.datetime.now()

    # Adversarial loss ground truths
    valid = np.ones((batch_size,) + disc_patch)
    fake = np.zeros((batch_size,) + disc_patch)

    for epoch in range(epochs):
        #print("Inside 1st for-loop")
        dataloader = DataLoader(dataset_name,img_res=(128, 128))  #(self, dataset_name, img_res=(128, 128))
        for batch_i, (imgs_A, imgs_B) in enumerate(dataloader.load_batch(batch_size)):

            # ----------------------
            #  Train Discriminators
            # ----------------------

            #print("Inside 2nd for loop")
            # Translate images to opposite domain
            fake_B = g_AB.predict(imgs_A)
            fake_A = g_BA.predict(imgs_B)

            # Train the discriminators (original images = real / translated = Fake)
            dA_loss_real = d_A.train_on_batch(imgs_A, valid)
            dA_loss_fake = d_A.train_on_batch(fake_A, fake)
            dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

            dB_loss_real = d_B.train_on_batch(imgs_B, valid)
            dB_loss_fake = d_B.train_on_batch(fake_B, fake)
            dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

            # Total disciminator loss
            d_loss = 0.5 * np.add(dA_loss, dB_loss)


            # ------------------
            #  Train Generators
            # ------------------

            # Train the generators
            g_loss = combined.train_on_batch([imgs_A, imgs_B],
                                                        [valid, valid,
                                                        imgs_A, imgs_B,
                                                        imgs_A, imgs_B])

            elapsed_time = datetime.datetime.now() - start_time

            # Plot the progress
            print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, epochs,
                                                                            batch_i, dataloader.n_batches,
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))

            # If at save interval => save generated image samples
            if batch_i % sample_interval == 0:
                sample_images(epoch, batch_i)
                d_A.save('d_A_model.h5')
                d_A.save_weights('d_A_save_weights.h5')
                d_B.save('d_B_model.h5')
                d_B.save_weights('d_B_save_weights.h5')
                g_AB.save('g_AB_model.h5')
                g_AB.save_weights('g_AB_save_weights.h5')
                g_BA.save('g_BA_model.h5')
                g_BA.save_weights('g_BA_save_weights.h5')
                combined.save('combined_model.h5')
                combined.save_weights('combined_save_weights.h5')

In [0]:
train(25, batch_size=1, sample_interval=200)

Train method
epochs is  25
dataset_name is  apple2orange


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/25] [Batch 0/995] [D loss: 0.325964, acc:  29%] [G loss: 9.358888, adv: 0.519217, recon: 0.382706, id: 0.367862] time: 0:00:00.297547 
dataset_name is  apple2orange


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


[Epoch 0/25] [Batch 1/995] [D loss: 0.379088, acc:  21%] [G loss: 5.701217, adv: 0.483703, recon: 0.212779, id: 0.237115] time: 0:00:01.744011 
[Epoch 0/25] [Batch 2/995] [D loss: 0.403324, acc:  10%] [G loss: 5.891314, adv: 0.411384, recon: 0.226426, id: 0.325058] time: 0:00:01.985200 
[Epoch 0/25] [Batch 3/995] [D loss: 0.433968, acc:   9%] [G loss: 5.318155, adv: 0.406136, recon: 0.201194, id: 0.340452] time: 0:00:02.227304 
[Epoch 0/25] [Batch 4/995] [D loss: 0.432953, acc:  14%] [G loss: 5.094115, adv: 0.429559, recon: 0.190456, id: 0.188552] time: 0:00:02.461026 
[Epoch 0/25] [Batch 5/995] [D loss: 0.365515, acc:  12%] [G loss: 5.235131, adv: 0.468191, recon: 0.192864, id: 0.215176] time: 0:00:02.702135 
[Epoch 0/25] [Batch 6/995] [D loss: 0.433024, acc:  15%] [G loss: 5.325734, adv: 0.422069, recon: 0.205265, id: 0.214029] time: 0:00:02.944980 
[Epoch 0/25] [Batch 7/995] [D loss: 0.395456, acc:  10%] [G loss: 5.207147, adv: 0.422066, recon: 0.194777, id: 0.243734] time: 0:00:03.

In [0]:
from keras.models import load_model
# Returns a compiled model identical to the previous one
d_A = load_model('/content/d_A_model.h5')
d_B = load_model('/content/d_B_model.h5')
g_AB = load_model('/content/g_AB_model.h5')
g_BA = load_model('/content/g_BA_model.h5')
combined = load_model('/content/combined_model.h5')

In [0]:
# Running the model for 25 epochs more 
train(10, batch_size=1, sample_interval=200)

In [0]:
################ Show the image ######################

import matplotlib.pyplot as plt
import os
import cv2
items = os.listdir('/content/images/apple2orange')   
print (items)    

for each_image in items:
  if each_image.endswith(".png"):
    print (each_image)
    full_path = "/content/images/apple2orange/" + each_image
    print (full_path)
    image = cv2.imread(full_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.figure()
plt.imshow(image)
plt.colorbar()
plt.axis('off')
#plt.grid(True)